# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import json
import pandas as pd
import pymongo
from datetime import datetime

In [2]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv",low_memory=True)
df_youtube.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


## Question 0

On regarde les types attribués nativement aux variables.

In [4]:
df_youtube.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7992 entries, 0 to 7991
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   video_id        7992 non-null   object 
 1   title           7992 non-null   object 
 2   channel_title   7992 non-null   object 
 3   category_id     7992 non-null   int64  
 4   tags            7992 non-null   object 
 5   views           7992 non-null   int64  
 6   likes           7992 non-null   int64  
 7   dislikes        7992 non-null   int64  
 8   comment_total   7992 non-null   int64  
 9   thumbnail_link  7992 non-null   object 
 10  date            7992 non-null   float64
dtypes: float64(1), int64(5), object(5)
memory usage: 686.9+ KB


### Nettoyer les données

On crée la variable tags_list contenant les tags sous forme de listes.

In [5]:
df_youtube["tags_list"]=df_youtube["tags"].apply(lambda x:x.split("|"))
df_youtube.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7992 entries, 0 to 7991
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   video_id        7992 non-null   object 
 1   title           7992 non-null   object 
 2   channel_title   7992 non-null   object 
 3   category_id     7992 non-null   int64  
 4   tags            7992 non-null   object 
 5   views           7992 non-null   int64  
 6   likes           7992 non-null   int64  
 7   dislikes        7992 non-null   int64  
 8   comment_total   7992 non-null   int64  
 9   thumbnail_link  7992 non-null   object 
 10  date            7992 non-null   float64
 11  tags_list       7992 non-null   object 
dtypes: float64(1), int64(5), object(6)
memory usage: 749.4+ KB


On merge df_youtube avec le fichier US_category_id.json pour récupérer le nom des catégories.             
Pour cela, on crée un dataframe category à partir des champs id et snippet.title de ce json.               
On merge ensuite category et df_youtube selon les category_id.                        

In [6]:
with open('./data/US_category_id.json', 'r') as f:
    category = json.load(f)

category = pd.json_normalize(category['items'])
category = category.drop(['kind','etag','snippet.channelId','snippet.assignable'],axis='columns')
category.columns=["category_id","category_name"]
category["category_id"]=category["category_id"].astype("int")
category.head()

,category_id,category_name
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


In [7]:
if "category_name" not in list(df_youtube.columns):
    df_youtube=df_youtube.merge(category,on="category_id",how="left")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,tags_list,category_name
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,"[logan paul vlog, logan paul, logan, paul, oly...",Entertainment
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09,"[Apple, iPhone 10, iPhone Ten, iPhone, Portrai...",Science & Technology


On regarde si video_id peut être utiliser comme identifiants des lignes.                     
Pour cela, une valeur donnée de video_id doit être associée à une seule ligne de df_youtube.

In [8]:
df_youtube.video_id.value_counts().max()==1

False

Dans le cas présent, video_id ne peut pas être utiliser comme identifiant.                          
On va donc utiliser l'index du dataframe comme identifiant pour les documents.

In [9]:
df_youtube=df_youtube.reset_index()
df_youtube.columns=[col if col!="index" else "_id" for col in df_youtube.columns]
df_youtube.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7992 entries, 0 to 7991
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   _id             7992 non-null   int64  
 1   video_id        7992 non-null   object 
 2   title           7992 non-null   object 
 3   channel_title   7992 non-null   object 
 4   category_id     7992 non-null   int64  
 5   tags            7992 non-null   object 
 6   views           7992 non-null   int64  
 7   likes           7992 non-null   int64  
 8   dislikes        7992 non-null   int64  
 9   comment_total   7992 non-null   int64  
 10  thumbnail_link  7992 non-null   object 
 11  date            7992 non-null   float64
 12  tags_list       7992 non-null   object 
 13  category_name   7992 non-null   object 
dtypes: float64(1), int64(6), object(7)
memory usage: 874.2+ KB


### Importer les données


On transforme chaque ligne de df_youtube en dictionnaire avec la fonction suivante.

In [10]:
def to_documents(df,col_list='all'):
    """
    Transformation des lignes du dataframe df en 
    plusieurs dictionnaires

    Args:
        df: nom du dataframe à convertir
        col_list: liste des variables à exporter

    Returns:
        liste de dictionnaires issus de la conversion
    """
    if type(col_list)==type([]) and len(col_list)!=0:
        df=df.copy()[col_list]
    DOCUMENTS=df.to_dict(orient='records')
    return DOCUMENTS

DOCUMENTS=to_documents(df_youtube,['_id','video_id','title','channel_title','category_name','tags','views','likes'])
print(len(DOCUMENTS),DOCUMENTS[0],sep=2*"\n")

7992

{'_id': 0, 'video_id': 'XpVt6Z1Gjjo', 'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!', 'channel_title': 'Logan Paul Vlogs', 'category_name': 'Entertainment', 'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life', 'views': 4394029, 'likes': 320053}


On insère ensuite les dictionnaires dans la collection.

In [11]:
collection.drop()
collection.insert_many(DOCUMENTS)
collection.find_one()

{'_id': 0,
 'video_id': 'XpVt6Z1Gjjo',
 'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_name': 'Entertainment',
 'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
 'views': 4394029,
 'likes': 320053}

## Question 1  

On va récupérer toutes les vidéos de la chaîne Apple.                            
On regroupe par title les documents ayant channel_title=='Apple' avec le requête suivante.

In [12]:
question1 = collection.aggregate([{"$match": {"channel_title":"Apple"}},
                                  {"$group" : {"_id" : "$title"}}])
question1=[video["_id"] for video in list(question1)]
print(question1)

set(question1)==set(list(df_youtube.query("channel_title=='Apple'").title.unique()))

['iPhone 8 Plus — Portraits of Her  — Apple', 'iPhone X — Introducing iPhone X — Apple', 'Apple Watch Series 3 + Apple Music — Roll — Apple']


True

## Question 2

On va compter le nombre de catégories différentes.                                        
On regroupe les documents par category_name et on renvoie la liste des category_name sans doublons avec la requête suivante.

In [13]:
question2=collection.aggregate([{"$group" : {"_id" : "$category_name"}}])
question2=sorted([cat["_id"] for cat in list(question2)])
print(question2,len(question2),sep=2*"\n")

set(question2)==set(df_youtube.category_name.unique())

['Autos & Vehicles', 'Comedy', 'Education', 'Entertainment', 'Film & Animation', 'Gaming', 'Howto & Style', 'Music', 'News & Politics', 'Nonprofits & Activism', 'People & Blogs', 'Pets & Animals', 'Science & Technology', 'Shows', 'Sports', 'Travel & Events']

16


True

## Question 3

On met les tags sous la forme d'une liste.                              
On crée pour chaque document, sa liste de tags à partir de la fonction map_reduce()

In [14]:
map_func = '''function(){emit(this._id,this.tags)}'''
reduce_func = '''function(key,values){return values.toString().split("|")}'''
out = '''{out:"tags_list"}'''
tags_list = collection.map_reduce(map_func,reduce_func,out)
tags_list = dict([(video["_id"],video["value"]) for video in list(tags_list.find())])

good_answer = df_youtube.tags_list.to_dict()
verif=True
for cat in tags_list.keys():
    if tags_list[cat]!=good_answer[cat]:
        verif=False
        break
print(verif)

True


On remplace ensuite les chaînes de caractères du champs tags par leurs équivalents en listes de chaînes de caractères.

In [15]:
for i in tags_list.keys(): 
    collection.update_one({"_id":i}, {"$set":{"tags": tags_list[i]}})
question3=list(collection.find({},{'title':1,'tags':1}))
question3[0]

{'_id': 0,
 'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'tags': ['logan paul vlog',
  'logan paul',
  'logan',
  'paul',
  'olympics',
  'logan paul youtube',
  'vlog',
  'daily',
  'comedy',
  'hollywood',
  'parrot',
  'maverick',
  'bird',
  'maverick clothes',
  'diamond play button',
  'logan paul diamond play button',
  '10M subscribers',
  'logan paul 1 year vlogging',
  '1 year vlog',
  'dwarf mamba play button',
  'logan paul history',
  'youtube history',
  '10M',
  '10M plaque',
  'youtube button',
  'diamond button',
  'logang',
  'logang 4 life']}

## Question 4

On va récupérer les vidéos les plus vues (les 5 meilleurs).                                    
On regroupe les documents par video_id et title, on calcule le nombre maximal de vues par vidéo et on les trie de manière décroissante selon ce nombre avec la requête suivante.

In [16]:
question4 = collection.aggregate([{"$group" : {"_id" : {"ID":"$video_id","title":"$title"}, 
                                               "maxViewsByVideo" : {"$max" : "$views"}}},
                                  {"$sort": {"maxViewsByVideo":-1}},
                                  {"$limit":5}])

question4=[video["_id"]["title"] for video in list(question4)]
print(question4)

good_answer=df_youtube.groupby(["video_id","title"]).views.max().reset_index()
good_answer=good_answer.sort_values("views",ascending=False).head().title
question4==list(good_answer)

["BTS (방탄소년단) 'DNA' Official MV", 'ZAYN - Dusk Till Dawn ft. Sia', 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher', 'Shakira - Perro Fiel (Official Video) ft. Nicky Jam', 'Star Wars: The Last Jedi Trailer (Official)']


True

## Question 5

On va calculer le nombre moyen de vues par catégorie de vidéos.                                    
On regroupe les documents par video_id et category_name, on calcule le nombre maximal de vues par vidéo.           
On regroupe par category_name les documents obtenus à l'étape précédente, on calcule le nombre moyen de vues par catégorie avec la requête suivante.

In [17]:
question5 = collection.aggregate([{"$group" : {"_id" : {"category":"$category_name","video":"$video_id"} ,
                                               "maxViewsByVideo" : {"$max" : "$views"}}},
                                  {"$group" : {"_id":"$_id.category",
                                               "averageViewsByCategory":{"$avg":"$maxViewsByVideo"}}}])
question5=dict([(project["_id"],project["averageViewsByCategory"]) for project in list(question5)])

good_answer=df_youtube.groupby(["category_name","video_id"]).views.max().reset_index()
good_answer=good_answer.groupby("category_name").views.mean()
print(good_answer,end=2*"\n")
verif=True
for cat in question5.keys():
    if question5[cat]!=good_answer.loc[cat]:
        print(cat)
        verif=False
        break
print(verif)

category_name
Autos & Vehicles         6.227279e+05
Comedy                   1.317201e+06
Education                5.505853e+05
Entertainment            1.059428e+06
Film & Animation         9.665473e+05
Gaming                   7.618923e+05
Howto & Style            5.560027e+05
Music                    1.444205e+06
News & Politics          5.335751e+05
Nonprofits & Activism    6.811782e+05
People & Blogs           8.901047e+05
Pets & Animals           6.339064e+05
Science & Technology     8.615636e+05
Shows                    8.726000e+03
Sports                   6.180747e+05
Travel & Events          4.264922e+05
Name: views, dtype: float64

True


## Question 6

On récupère les chaines Youtube avec la plus grande moyenne de likes (les 5 meilleurs).                           
On regroupe les documents par channel_title et video_id, on calcule le nombre maximal de likes par vidéo.        
On regroupe par channel_title les documents obtenus à l'étape précédente, on calcule la moyenne de likes par chaine puis on les trie de manière décroissante selon ce nombre.   
On renvoie ensuite les 5 premières chaines avec la requête suivante.

In [18]:
question6=collection.aggregate([{"$group" : {"_id" : {"channel":"$channel_title","video":"$video_id"}, 
                                             "maxLikesByVideo" : {"$max" : "$likes"}}},
                                {"$group" : {"_id" : "$_id.channel", 
                                             "averageLikesByChannel" : {"$avg" : "$maxLikesByVideo"}}},
                                {"$sort": {"averageLikesByChannel":-1}},
                                {"$limit":5}])
question6=[project["_id"] for project in list(question6)]
print(question6)

good_answer=df_youtube.groupby(["channel_title","video_id"]).likes.max().reset_index()
good_answer=good_answer.groupby("channel_title").likes.mean().reset_index()
good_answer=good_answer.sort_values("likes",ascending=False).head()
question6==list(good_answer.channel_title)

['ibighit', 'ZaynVEVO', 'BETNetworks', 'melanie martinez', 'jypentertainment']


True